# Datasets Preprocessing

This Notebook contains the preprocessing of the raw datasets after being downloaded from the web. Our benchmarks and scripts expects a `.bin` file with only the doubles (no column header) in their IEEE 754 (`double`) representation. This notebook transforms each of the 30 datasets into this format.

- Version: 0.0.1     
- Last Edited: 15/04/2023

## Dependencies
To use this notebook you need Python (>3.7), Jupyter Notebook and the Libraries `pandas` and `numpy` installed (https://pandas.pydata.org/docs/). To install them, execute the next two cell.

In [ ]:
%pip install pandas

In [ ]:
%pip install numpy

In [5]:
import pandas as pd
import numpy as np
import os

# NEON Datasets

When downloaded and unzipped, these datasets are scattered across many directories and files. The following cells process every directory structure to build the dataset

## PM10-Dust
Download URL: https://doi.org/10.48443/4E6X-V373

In [47]:
df = pd.DataFrame({'PM10Median': []})
root = './NEON_size-dust-particulate/'
dates = os.listdir(root)
dates.sort()
for date in dates:
    curr_path = root + date + '/'
    if not os.path.isdir(curr_path): continue
    enter = False
    for filename in os.listdir(curr_path):
        if '30_minutes' in filename and filename.endswith('.csv') and ('basic.2020' in filename or 'basic.2021' in filename):
            enter = True
            filename_path = curr_path + filename
            df_tmp = pd.read_csv(filename_path)
            df_tmp = df_tmp[['PM10Median']].dropna()
            df = pd.concat([df, df_tmp], copy=False)
    if enter == False:
        print(curr_path)
df['PM10Median'].values.tofile('neon_pm10_dust.bin')


## Dew-Point-Temp

Download URL: https://doi.org/10.48443/Z99V-0502

In [45]:
df = pd.DataFrame({'dewTempMean': []})
root = './NEON_rel-humidity-buoy/'
dates = os.listdir(root)
dates.sort()
for date in dates:
    curr_path = root + date + '/'
    if not os.path.isdir(curr_path): continue
    enter = False
    for filename in os.listdir(curr_path):
        if '1min' in filename and filename.endswith('.csv') and ('basic.2020' in filename or 'basic.2021' in filename):
            enter = True
            filename_path = curr_path + filename
            df_tmp = pd.read_csv(filename_path)
            df_tmp = df_tmp[['dewTempMean']].dropna()
            df = pd.concat([df, df_tmp], copy=False)
    if enter == False:
        print(curr_path)
df['dewTempMean'].values.tofile('neon_dew_point_temp.bin')


## Air Pressure

Download URL: https://doi.org/10.48443/RXR7-PP32

In [44]:
df = pd.DataFrame({'staPresMean': []})
root = './NEON_pressure-air/'
dates = os.listdir(root)
dates.sort()
for date in dates:
    curr_path = root + date + '/'
    if not os.path.isdir(curr_path): continue
    enter = False
    for filename in os.listdir(curr_path):
        if '1min' in filename and filename.endswith('.csv') and ('basic.2020' in filename or 'basic.2021' in filename):
            enter = True
            filename_path = curr_path + filename
            df_tmp = pd.read_csv(filename_path)
            df_tmp = df_tmp[['staPresMean']].dropna()
            df = pd.concat([df, df_tmp], copy=False)
    if enter == False:
        print(curr_path)
df['staPresMean'].values.tofile('neon_air_pressure.bin')


## Wind Direction

Download URL: https://doi.org/10.48443/S9YA-ZC81

In [43]:
df = pd.DataFrame({'windDirMean': []})
root = './NEON_wind-2d/'
dates = os.listdir(root)
dates.sort()
for date in dates:
    curr_path = root + date + '/'
    if not os.path.isdir(curr_path): continue
    enter = False
    for filename in os.listdir(curr_path):
        if '2min' in filename and filename.endswith('.csv') and ('basic.2020' in filename or 'basic.2021' in filename):
            enter = True
            filename_path = curr_path + filename
            df_tmp = pd.read_csv(filename_path)
            df_tmp = df_tmp[['windDirMean']].dropna()
            df = pd.concat([df, df_tmp], copy=False)
    if enter == False:
        print(curr_path)
df['windDirMean'].values.tofile('neon_wind_dir.bin')


## IR-Bio-Temp

Download URL: https://doi.org/10.48443/JNWY-B177

In [41]:
df = pd.DataFrame({'bioTempMean': []})
root = './NEON_temp-bio/'
dates = os.listdir(root)
dates.sort()
for date in dates:
    curr_path = root + date + '/'
    if not os.path.isdir(curr_path): continue
    enter = False
    for filename in os.listdir(curr_path):
        if '1_minute' in filename and filename.endswith('.csv') and ('basic.2020' in filename or 'basic.2021' in filename):
            enter = True
            filename_path = curr_path + filename
            df_tmp = pd.read_csv(filename_path)
            df_tmp = df_tmp[['bioTempMean']].dropna()
            df = pd.concat([df, df_tmp], copy=False)
    if enter == False:
        print(curr_path)
df['bioTempMean'].values.tofile('neon_bio_temp.bin')


# STOCKS Datasets

When downloaded and unzipped, these datasets are scattered across many directories and files. The following cells process every directory structure to build the dataset

## Stocks USA / DE / UK

Download URL: https://zenodo.org/record/3886895#%23.ZDBBKuxBz0r

- Create the directory with the name inside the variable `tmp_dir`. We will save temporary CSVs on this directory that will then be joined (we do this to optimize the process)

In [56]:
countries = ['Germany', 'United Kingdom', 'USA']
tmp_dir = 'stocks_tmp'

In [58]:
for country in countries:
    file_prefix = 'Stocks ' + country
    df = pd.DataFrame({2: []})
    root = './FinancialData/quotes/'
    dates = os.listdir(root)
    dates.sort()
    x = 0
    for date in dates:
        df_date = pd.DataFrame({2: []})
        if not date.isdigit(): continue
        curr_path = root + date + '/'
        print(date)
        for filename in os.listdir(curr_path):
            if filename.startswith(file_prefix) and filename.endswith('.zip'):
                filename_path = curr_path + filename
                df_tmp = pd.read_csv(filename_path, compression='zip', header=None)
                df_tmp = df_tmp[[2]]
                df_date = pd.concat([df_date, df_tmp], copy=False)
        df_date.to_csv('./' + tmp_dir + '/stocks_' + country + '_' + date + ".csv", index=False, header=['value'])
        if x == 3:
            break
        x+=1
    dates = os.listdir(tmp_dir)
    dates.sort()
    df_list = []
    for date in dates:
        if country not in date:
            continue
        curr_path = tmp_dir + '/' + date
        df = pd.read_csv(curr_path)
        df_list.append(df)
    df_final = pd.concat(df_list)
    df_final['value'].values.tofile('stocks_' + country + '.bin')

01012019
01022019
01032019
01042019
01012019
01022019
01032019
01042019
01012019
01022019
01032019
01042019


# Public BI Benchmark
Reference: https://github.com/cwida/public_bi_benchmark

## CommonGov
Download URL: https://homepages.cwi.nl/~boncz/PublicBIbenchmark/CommonGovernment/

Download each of the files inside the URL

In [13]:
gov1 = pd.read_csv('./CommonGovernment_1.csv.bz2', header=None, sep='|', usecols=[9, 25, 29, 30, 39])
gov2 = pd.read_csv('./CommonGovernment_2.csv.bz2', header=None, sep='|', usecols=[9, 25, 29, 30, 39])
gov3 = pd.read_csv('./CommonGovernment_3.csv.bz2', header=None, sep='|', usecols=[9, 25, 29, 30, 39])
gov4 = pd.read_csv('./CommonGovernment_4.csv.bz2', header=None, sep='|', usecols=[9, 25, 29, 30, 39])
gov5 = pd.read_csv('./CommonGovernment_5.csv.bz2', header=None, sep='|', usecols=[9, 25, 29, 30, 39])
gov6 = pd.read_csv('./CommonGovernment_6.csv.bz2', header=None, sep='|', usecols=[9, 25, 29, 30, 39])
gov7 = pd.read_csv('./CommonGovernment_7.csv.bz2', header=None, sep='|', usecols=[9, 25, 29, 30, 39])
gov8 = pd.read_csv('./CommonGovernment_8.csv.bz2', header=None, sep='|', usecols=[9, 25, 29, 30, 39])
gov9 = pd.read_csv('./CommonGovernment_9.csv.bz2', header=None, sep='|', usecols=[9, 25, 29, 30, 39])
gov10 = pd.read_csv('./CommonGovernment_10.csv.bz2', header=None, sep='|', usecols=[9, 25, 29, 30, 39])
gov11 = pd.read_csv('./CommonGovernment_11.csv.bz2', header=None, sep='|', usecols=[9, 25, 29, 30, 39])
gov12 = pd.read_csv('./CommonGovernment_12.csv.bz2', header=None, sep='|', usecols=[9, 25, 29, 30, 39])
gov13 = pd.read_csv('./CommonGovernment_13.csv.bz2', header=None, sep='|', usecols=[9, 25, 29, 30, 39])
gov = pd.concat([gov1, gov2, gov3, gov4, gov5, gov6, gov7, gov8, gov9, gov10, gov11, gov12, gov13])
gov = gov.dropna()
gov[9].values.tofile('gov10.bin')
gov[25].values.tofile('gov26.bin')
gov[29].values.tofile('gov30.bin') 
gov[30].values.tofile('gov31.bin') 
gov[39].values.tofile('gov40.bin')

# Medicare
Download URL: https://homepages.cwi.nl/~boncz/PublicBIbenchmark/Medicare3/

In [9]:
medicare1 = pd.read_csv('./Medicare3_1.csv.bz2', header=None, sep='|', usecols=[0, 8], dtype=np.float64)
medicare = medicare1.dropna()
medicare[0].values.tofile('medicare1.bin')
medicare[8].values.tofile('medicare9.bin')

# CMSProvider

Download URL: https://homepages.cwi.nl/~boncz/PublicBIbenchmark/CMSprovider/   

Download each of the files inside the URL

In [6]:
cms1 = pd.read_csv('./CMSprovider_1.csv.bz2', header=None, sep='|', usecols=[0, 8, 24], dtype=np.float64)
cms2 = pd.read_csv('./CMSprovider_2.csv.bz2', header=None, sep='|', usecols=[0, 8, 24], dtype=np.float64)
cms = pd.concat([cms1, cms2])
cms = cms.dropna()
cms[0].values.tofile('cms1.bin')
cms[8].values.tofile('cms9.bin')
cms[24].values.tofile('cms25.bin') 

# NYC

Download URL: https://homepages.cwi.nl/~boncz/PublicBIbenchmark/NYC/

Download each of the files inside the URL

In [2]:
nyc1 = pd.read_csv('./NYC_1.csv.bz2', header=None, sep='|', usecols=[28])
nyc2 = pd.read_csv('./NYC_2.csv.bz2', header=None, sep='|', usecols=[28])
nyc = pd.concat([nyc1, nyc2])
nyc = nyc.dropna()
nyc[28].values.tofile('nyc29.bin')

# Arade

Download URL: https://homepages.cwi.nl/~boncz/PublicBIbenchmark/Arade/   

Download each of the files inside the URL

In [23]:
arade = pd.read_csv('./Arade_1.csv.bz2', sep='|', header=None)
arade[3].values.tofile('arade4.bin')

# The Other Datasets

# POI (Lat - Lon)

Download URL: https://www.kaggle.com/datasets/ehallmar/points-of-interest-poi-database

In [21]:
poi = pd.read_csv('./poi.csv')
poi['latitude_radian'].values.tofile('poi_lat.bin')
poi['longitude_radian'].values.tofile('poi_lon.bin')

,name,latitude_radian,longitude_radian,num_links,links,num_categories,categories
0,"YAYCHI, WEST AZERBAIJAN",0.683175,0.778053,13,Baba Jik Rural District; West Azerbaijan Provi...,1,POPULATED PLACES IN CHALDORAN COUNTY
1,MOUNT FISKE GLACIER,0.648196,-2.071114,9,Mount Fiske; Mount Warlow Glacier; U.S. state;...,3,GLACIERS OF THE SIERRA NEVADA (U.S.); GLACIERS...
2,ALATONA,0.258356,-0.103606,10,Diabaly; Alatona Irrigation Project; Mali; Nio...,2,POPULATED PLACES IN SÉGOU REGION; IRRIGATION P...
3,PEMBA ISLAND,-0.090175,0.694350,43,Malaysia Airlines Flight 370; Arusha; Chake Ch...,5,PEMBA ISLAND; ISLANDS OF TANZANIA; ISLANDS OF ...
4,MBOLO,0.149517,0.359829,6,UTC; Sub-prefectures of the Central African Re...,2,N'DÉLÉ; POPULATED PLACES IN BAMINGUI-BANGORAN
...,...,...,...,...,...,...,...
424200,"PITTSFIELD TOWNSHIP, LORAIN COUNTY, OHIO",0.719842,-1.434913,28,"Oberlin, Ohio; List of sovereign states; Civil...",1,"TOWNSHIPS IN LORAIN COUNTY, OHIO"
424201,KEVIN BARTLETT RESERVE,-0.660297,2.531149,9,"Burnley, Victoria; Collingwood City F.C.; Rich...",2,SOCCER VENUES IN AUSTRALIA; SPORTS VENUES IN M...
424202,"WEST SPRINGFIELD, VIRGINIA",0.676984,-1.347966,35,"Fairfax County, Virginia; List of sovereign st...",4,"CENSUS-DESIGNATED PLACES IN FAIRFAX COUNTY, VI..."
424203,GLEN ROCK HISTORIC DISTRICT,0.694515,-1.339240,6,"York County, Pennsylvania; Historic district (...",4,HISTORIC DISTRICTS ON THE NATIONAL REGISTER OF...


## Food Prices

Download URL: https://data.humdata.org/dataset/wfp-food-prices

In [ ]:
food_prices_df = pd.read_csv('wfpvam_foodprices.csv')
food_prices_df['mp_price'].values.tofile('food_prices.bin')

## Bird-migration

Download URL: https://github.com/influxdata/influxdb2-sample-data/blob/master/bird-migration-data/bird-migration.csv

In [50]:
bird_migration_df = pd.read_csv('./bird_migration.csv', skiprows=3)
bird_migration_df['_value'].values.tofile('bird_migration_f.bin')

## Bitcoin-price

Download URL: https://raw.githubusercontent.com/influxdata/influxdb2-sample-data/master/bitcoin-price-data/bitcoin-historical-annotated.csv

In [54]:
bitcoin_price_df = pd.read_csv('./bitcoin.csv', skiprows=3)
bitcoin_price_df['_value'].values.tofile('bitcoin_f.bin')

## Bitcoin Transaction

Download URL: https://gz.blockchair.com/bitcoin/transactions/

Search and download for day 2022/03/26, or directly download using this link:
https://gz.blockchair.com/bitcoin/transactions/blockchair_bitcoin_transactions_20220326.tsv.gz

In [15]:
bitcoin_tr_df = pd.read_csv('./bitcoin_transactions.tsv.gz', delimiter='\t', compression='gzip')
bitcoin_tr_df['output_total_usd'].values.tofile('bitcoin_transactions_f.bin')

## SSD-HDD Benchmark

Download URL: https://www.kaggle.com/datasets/alanjo/ssd-and-hdd-benchmarks

In [14]:
ssd_df = pd.read_csv('SSD_HDD_benchmarks_v9.csv')
ssd_df['diskCapacity'].values.tofile('ssd_hdd_benchmarks_f.bin')

## City Temperature

Download URL: https://www.kaggle.com/datasets/sudalairajkumar/daily-temperature-of-major-cities

In [ ]:
city_temp_df = pd.read_csv('city_temperature.csv.gz', compression='gzip', header=None)
city_temp_df[2].values.tofile('city_temperature_f.bin')

## Basel-Temp & Wind

Download URL: https://www.meteoblue.com/en/weather/archive/export/basel_switzerland

- Date filters: 2008-01-01 until 2022-01-31   
- Boxes selected: Temperature and Wind Speed (10m)

In [12]:
basel = pd.read_csv('dataexport_20230415T103128.csv', header=None, skiprows=10)
basel_wind = basel[2].values.tofile('basel_wind_t.bin')
basel_temp = basel[1].values.tofile('basel_temp.bin')